Remaining problems:
* index of target word in sentences, where punctuation is not separated (tokenized), e.g. "jag tycker att relationen till min sambos ursprung berikar mig enormt!"
* compounds, e.g., "globalistelit"; tokeniser does not necesserily identigy boundery for `globalist|elit`; if it does, this would solve it
* more than one target in sentence (consider `sent.split(" ").index(word)`)
    * use `re` for strings somehow??
* what is the difference between `T5TokenizerFast` and `T5Tokenizer` other than the latter lacks a method for `word_ids()`? 

In [1]:
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
import re
from collections import Counter
from pathlib import Path
from difflib import SequenceMatcher
import os

In [2]:
# from transformers import pipeline
# from transformers import AutoModel, AutoTokenizer

In [3]:
# https://huggingface.co/transformers/v4.1.1/main_classes/tokenizer.html#transformers.PreTrainedTokenizerFast
from transformers import T5TokenizerFast, T5EncoderModel

In [4]:
# T5tokenizer = T5TokenizerFast.from_pretrained("t5-small")
# T5model = T5EncoderModel.from_pretrained("t5-small")
# T5tokenizer = T5TokenizerFast.from_pretrained("t5-large", model_max_length=512)
# T5model = T5EncoderModel.from_pretrained("t5-large")

In [5]:
# counter = Counter()
# path = Path("../files")
# i = 0
# for file in os.listdir(path):
#     with open(path / file, encoding="utf-8") as f:
#         for line in f:
#             line = line.split("\t")
#             counter.update(line[0].split("; "))
#             print(i, end="\r")
#             i += 1

In [6]:
#sorted([k for k in counter.keys() if not k.startswith("X_hjälpa")])

In [7]:
# sorted([k for k in counter.keys() if k.startswith("X_hjälpa")])

In [8]:
ignore = tuple(["P1_", "V1_hjälpa", "V2_hjälpa", "X_hjälpa"])

In [9]:
# sorted([k for k in counter.keys() if not k.startswith(ignore)])

In [10]:
def p_split(string):
    k, v = (f"{string.split(' -> ')[1]}_{string.split()[-1]}", string.split(' -> ')[2])
    return k,v

In [11]:
#with open("/home/max/Documents/mlt/thesis/dw_change/data/utils/dwts.paradigm") as f:
with open("dwts.paradigm.txt", encoding="utf-8") as f:
    paradigms = [line.strip("\n") for line in f.readlines() if not line.startswith("#")]
paradigms = [p for p in paradigms if p != ""]
paradigms = [p.split(" #")[0] for p in paradigms]
paradigms = dict([p_split(paradigm) for paradigm in paradigms])
print(paradigms)

{'N1_förortsgäng': '\\bförortsgäng(|et|en|s|ets|ens)\\b', 'N1C_förortsgängX': '\\b(förortsgäng)s?((?!\\b|et|en|s\\b).*?)\\b', 'N1_återvandring': '\\båtervandring(|en|ar|s|ens|ars|arna|arnas)\\b', 'N2_återvandrare': '\\båtervandrar(e|en|es|ens|na|nas)\\b', 'N1C_återvandringsX': '\\b(återvandring)s?((?!\\b|en|ar|s\\b).*?)\\b', 'N2C_återvandrarX': '\\b(återvandrar)((?!\\b|e\\b|en|es|na).*?)\\b', 'V1_återvandra': '\\båtervandra(\\b|r|d|de|t|nde|ndet)\\b', 'N1_berikare': '\\bberikar(e|en|na|es|ens|nas)\\b', 'N1C_berikareX': '\\b(berikar)((?!\\b|e\\b|en|na|es).*?)\\b', 'V1_berika': '\\bberik(a|ar|ad|ade|at|ande|as|ats|ades)\\b', 'N1_kulturberikare': '\\bkulturberikar(e|en|na|es|ens|nas)\\b', 'N1C_kulturberikarX': '\\b(kulturberikar)((?!\\b|e\\b|en|na|es).*?)\\b', 'V1_kulturberika': '\\bkulturberik(a|ar|ad|ade|at|ande|as|ats|ades)\\b', 'P1_ordning_och_reda_i_flyktingpolitiken': 'ordning och reda i flyktingpolitiken', 'N1_globalist': '\\bglobalist(|en|er|erna|s|ens|ers|ernas)\\b', 'N1C_globali

In [12]:
def loop_for_idx(sentence):
    encoded = tokenizer.encode_plus(sentence, return_tensors="pt")
    inbetween = tokenizer.decode(encoded["input_ids"][0], skip_special_tokens = True, clean_up_tokenization_spaces=True)
    return inbetween

In [13]:
def get_word_vector(sentence, exact_match, lemma, model, device = "cpu", only_check = True):
    if lemma.startswith("X"): # X_globalist
        true_lemma = lemma.split("_")[-1] 
        true_wf = true_lemma + exact_match.split(true_lemma)[-1]
    else: 
        if lemma.endswith("X"): # N1C_globalistX
            true_lemma = lemma.split("_")[1][:-1]
            true_wf = lemma.split("_")[1][:-1] #true_lemma
        else: # N1_globalist
            true_lemma = lemma.split("_")[1]
            true_wf = exact_match

    # sentence=loop_for_idx(sentence)
    encoded = tokenizer.encode_plus(sentence, return_tensors="pt", truncation=True, max_length=512)
    #print(encoded)
    tokens = encoded.tokens() #[tokenizer.decode(wid) for wid in encoded["input_ids"][0]]

    # short_tokens = "".join([tok.replace("▁", " ") for tok in tokens]).replace("</s>", "")

    # if exact_match not in short_tokens.split():
    #     print()
    #     print(sentence.replace(exact_match, exact_match.upper()))
    #     #print(short_tokens)
    #     encoded = tokenizer.encode_plus(sentence, return_tensors="pt", truncation="only_second", max_length=512)
    #     tokens = encoded.tokens() #[tokenizer.decode(wid) for wid in encoded["input_ids"][0]]
    #     sentence = "".join([tok.replace("▁", " ") for tok in tokens]).replace("</s>", "")

    try:
        idx = sentence.split().index(exact_match) # will not match tokenizer; hence `map_tok()`
        # inbetween = tokenizer.decode(encoded["input_ids"][0], skip_special_tokens = True, clean_up_tokenization_spaces=True)
        
        #print(inbetween)
        # idx = inbetween.split().index(exact_match)
    except:
        print("Oops! `exact_match` not in sentence. |", lemma, "|", exact_match, "|", sentence)
        return
    
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)[0]
    if token_ids_word.tolist() == []:
        return
    #print(token_ids_word)
    
    if lemma.endswith("X") or lemma.startswith("X"):
        start_with = min(token_ids_word)
        outer = start_with
        top    = 0

        if lemma.startswith("X"):
            for i, idx in enumerate(token_ids_word):
                if true_lemma.startswith(tokens[idx].replace("▁", "")):
                    token_ids_word = token_ids_word[i:]
                    start_with = min(token_ids_word)
                    break

        for i in token_ids_word: 
            i = i + 1
            candidate = "".join([tok.replace("▁", "") for tok in tokens[min(token_ids_word):i]])
            score = SequenceMatcher(None, true_wf, candidate).ratio()
            if score >= top:
                top = score
                outer = i

        token_ids_word = np.arange(start_with, outer) # arrange
        #print(token_ids_word)

    if only_check:
        #print(" ".join(tokens))#, end = "\r")
        try:
            tokens = tokens[token_ids_word[0]:token_ids_word[-1]+1]
        except:
            print("\nERROR: No `token_ids_word` |", lemma, token_ids_word, "|", " ".join(encoded.tokens()))
            print(sentence)
            return
        tokens_short = "".join([tok.replace("_", "") for tok in tokens])
        if not "hjälpa_på" in lemma:
            if tokens_short != true_wf:
                if sum([dwe in tokens_short for dwe in ["globali", "berika", "återvandr", "förortsgäng"]]) == 0: # note short form for globalist (globali) due to t5 tokenization
                    print(f"{lemma} | {true_lemma} | {exact_match} | {true_wf}  >>> {' '.join(tokens)} <<<  ({token_ids_word}):\n1:{sentence})\n2:{' '.join(encoded.tokens())}")
        return

    encoded.to(device)
    # with torch.no_grad():
    #     output = model(**encoded)

    with torch.no_grad():
        output = model.encoder(
            input_ids=encoded["input_ids"], 
            attention_mask=encoded["attention_mask"], 
            return_dict=True
        )

    last_hidden = output.last_hidden_state.squeeze()
    word_tokens_output = last_hidden[token_ids_word]

    return word_tokens_output.mean(dim=0)


In [14]:
def map_tok(sentence):
    # sentence = re.sub(r" ([\.,!?])", r"\1", sentence)
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("+", " + ")
    sentence = sentence.replace("&", " & ")
    sentence = sentence.replace("@", " @ ")
    sentence = sentence.replace(":", " : ")
    sentence = sentence.replace("*", " * ")
    sentence = sentence.replace("^", " ^ ")
    sentence = sentence.replace("ü", "u")
    sentence = sentence.replace("$", "s")
    # sentence = sentence.replace(">", "")
    # sentence = re.sub(r"([a-zåäö]):([a-zåäö])", r"\1 : \2", sentence)
    sentence = sentence.replace("=) ", "")
    sentence = sentence.replace(">= ", "")
    sentence = sentence.replace("=>", "")
    sentence = sentence.replace(">>", ' " ')
    sentence = sentence.replace("<<", ' " ')
    sentence = sentence.replace("''", ' " ')
    sentence = sentence.replace("´", " ' ") # see below
    sentence = sentence.replace("(", " ( ") # see below
    sentence = sentence.replace(")", " ) ") # see below
    sentence = sentence.replace("| ", "")
    sentence = sentence.replace("överraskningsberika", "överrasknings berika")
    sentence = sentence.replace("\U0001F923", "")
    sentence = re.sub(r"([a-zåäö])(['`])([a-zåäö])", r"\1 \2 \3", sentence)
    sentence = re.sub(r"([()\?=%!<>~«])([a-zåäö0-9])", r"\1 \2", sentence)
    sentence = re.sub(r"([a-zåäö0-9])([()\?=%!<>~«])", r"\1 \2", sentence)
    sentence = re.sub(r"([0-9]),([0-9])", r"\1 , \2", sentence)
    sentence = re.sub(r"#+", " * ", sentence)
    sentence = re.sub(r"([=¤])+", r"\1", sentence)
    sentence = re.sub(r" +", " ", sentence)
    return sentence

In [15]:
def get_word_embeddings(
    model,
    directory,
    vector_dir,
    paradigms,
    ignore,
    device="cpu",
    only_check = True,
    dev_restriction=None,
    re_start = None
):

    directory = Path(directory)
    vector_dir = Path(vector_dir)

    isExist = os.path.exists(vector_dir)
    if not isExist:
        os.makedirs(vector_dir)  
    
    files = os.listdir(directory)
    if re_start != None:
        years = sorted([int(y.replace(".txt", "")) for y in files])
        files = [f"{y}.txt" for y in years if y >= re_start]
    
    if dev_restriction != None:
        files = dev_restriction

    model.to(device)

    for file in files:
        print()
        print(file)
        with open(directory / file, encoding="utf-8") as f, open(vector_dir / file, "w", encoding="utf-8") as out:
            for i, line in enumerate(f):
                if i % 10 == 0:
                    print(i, end="\r")
                lemma, n, sentence = tuple(line.strip("\n").split("\t"))

                sentence = map_tok(sentence)

                if int(n) == 1: 
                    if lemma.startswith(ignore):
                        continue
                    if lemma in paradigms:
                        regex = paradigms[lemma]
                        regex = re.compile(regex)    
                        exact_match = re.search(regex, sentence)
                        if exact_match == None:
                            print("ERROR:", lemma, "||", sentence)
                        exact_match = exact_match.group()                        
                    else:
                        regex = re.compile(f"\\b[0-9a-zåäö]*{lemma.split('_')[-1]}.*?\\b")
                        exact_match = re.search(regex, sentence)
                        if exact_match == None:
                            print("ERROR:", lemma, "|", regex, "|", sentence)
                        exact_match = exact_match.group()
                    vector = get_word_vector(sentence, exact_match, lemma, model, device, only_check)
                    if only_check or vector == None:
                        continue
                        
                    vector = " ".join([str(v) for v in vector.tolist()]) # consider torch.save(tensor, 'file.pt')
                    out.write(f"{lemma}\t{vector}\n")                   

                else: # two instances of the same lemma = problem
                    for l in lemma.split("; "):
                        if l.startswith(ignore):
                            continue                        
                        if l in paradigms:
                            regex = paradigms[l]
                            regex = re.compile(regex)    
                            exact_match = re.search(regex, sentence).group() 
                        else:
                            regex = re.compile(f"\\b[0-9a-zåäö]*{l.split('_')[-1]}.*?\\b")
                            exact_match = re.search(regex, sentence).group()
                        vector = get_word_vector(sentence, exact_match, l, model, device, only_check)
                        if only_check or vector == None:
                            continue
                        vector = " ".join([str(v) for v in vector.tolist()])
                        out.write(f"{l}\t{vector}\n")

In [16]:
model_name = "google-t5/t5-3b" # 't5-small' # "google-t5/t5-3b"
# model_name = "google-t5/t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name, model_max_length=512)
model = T5EncoderModel.from_pretrained(model_name)
short_name = model_name.split("/")[-1]

In [17]:
%%time
get_word_embeddings(
    model=model, 
    directory=Path("../data/corpus/fb_pol_files/"), #"C:\Users\xbohma\Desktop\work\work\data\corpus\files"
    vector_dir=f"../data/vectors/fb_pol/{short_name}", #"../bert-base-swedish-cased/", # "C:\Users\xbohma\Desktop\work\work\data\vectors\bert-base-swedish-cased"
    paradigms=paradigms,
    ignore=ignore,
    device="cuda",
    only_check = False,
    #dev_restriction = ["2018.txt"]
    #re_start = 2022
)


2000.txt
120
2001.txt
60
2002.txt
40
2003.txt
30
2004.txt
120
2005.txt
180
2006.txt
520
2007.txt
1580
2008.txt
3780
2009.txt
4120
2010.txt
5060
2011.txt
4360
2012.txt
4600
2013.txt
4140
2014.txt
3800
2015.txt
3730
2016.txt
4840
2017.txt
5130
2018.txt
11570
2019.txt
8640
2020.txt
7530
2021.txt
7120
2022.txt
Oops! `exact_match` not in sentence. | N1C_globalistX | globalistetna | storbritanien o indien har idag kanske inte sa mycket att gora med varandra , dock sa hoppas sa klart imperialisterna（globalistetna）att ha indien pa sin sida
CPU times: total: 1h 38min 37s
Wall time: 1h 31min 51s


In [ ]:
# %%time
# get_word_embeddings(
#     model=model, 
#     directory=Path("../data/corpus/files"), #"C:\Users\xbohma\Desktop\work\work\data\corpus\files"
#     vector_dir=f"../data/vectors/{short_name}", #"../bert-base-swedish-cased/", # "C:\Users\xbohma\Desktop\work\work\data\vectors\bert-base-swedish-cased"
#     paradigms=paradigms,
#     ignore=ignore,
#     device="cpu",
#     only_check = True,
#     #dev_restriction = ["2018.txt"]
#     #re_start = 2022
# )